In [1]:
!pip install pandas numpy matplotlib seaborn wordcloud nltk keras tensorflow


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#import libraries
import json
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Conv1D, SimpleRNN, Bidirectional,MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D, LSTM, GRU
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.layers import Dense, LSTM, Embedding
from tensorflow import keras
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# from sklearn.utils.class_weight import compute_class_weight

In [5]:
# Load the JSON file into a pandas DataFrame
df = pd.read_csv('News_Category_Dataset.csv')
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [6]:
# #dropping unnecessary columns
new_df = df.drop(columns=['authors','link','date'])
new_df.head()
print(len(new_df))

209527


# New Section

In [7]:
# create final dataframe of combined headline and short_description
final_df = new_df.copy()
final_df['length_of_news'] = final_df['headline'] + final_df['short_description']
final_df.drop(['headline','short_description'], inplace=True, axis=1)
final_df['len_news'] = final_df['length_of_news'].apply(lambda x: len(str(x)))
final_df.head()


,category,length_of_news,len_news
0,U.S. NEWS,Over 4 Million Americans Roll Up Sleeves For O...,230
1,U.S. NEWS,"American Airlines Flyer Charged, Banned For Li...",248
2,COMEDY,23 Of The Funniest Tweets About Cats And Dogs ...,133
3,PARENTING,The Funniest Tweets From Parents This Week (Se...,215
4,U.S. NEWS,Woman Who Called Cops On Black Bird-Watcher Lo...,233


In [8]:
# final_df.drop(columns=['category'])
final_df.head()

,category,length_of_news,len_news
0,U.S. NEWS,Over 4 Million Americans Roll Up Sleeves For O...,230
1,U.S. NEWS,"American Airlines Flyer Charged, Banned For Li...",248
2,COMEDY,23 Of The Funniest Tweets About Cats And Dogs ...,133
3,PARENTING,The Funniest Tweets From Parents This Week (Se...,215
4,U.S. NEWS,Woman Who Called Cops On Black Bird-Watcher Lo...,233


In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DhanushDevadiga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# Preprocess the data
def preprocess_text(text):



#     # Remove special characters and digits
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Split into words
    words = text.split()
    # Join the words
    preprocessed_text = ' '.join(words)
    return preprocessed_text

final_df['length_of_news'] = final_df['length_of_news'].apply(lambda x: preprocess_text(str(x)))


In [11]:
final_df['category_merged']=final_df['category'].replace({"HEALTHY LIVING": "WELLNESS",
"QUEER VOICES": "GROUPS VOICES",
"BUSINESS": "BUSINESS & FINANCES",
"PARENTS": "PARENTING",
"BLACK VOICES": "GROUPS VOICES",
"THE WORLDPOST": "WORLD NEWS",
"STYLE": "STYLE & BEAUTY",
"GREEN": "ENVIRONMENT",
"TASTE": "FOOD & DRINK",
"WORLDPOST": "WORLD NEWS",
"SCIENCE": "SCIENCE & TECH",
"TECH": "SCIENCE & TECH",
"MONEY": "BUSINESS & FINANCES",
"ARTS": "ARTS & CULTURE",
"COLLEGE": "EDUCATION",
"LATINO VOICES": "GROUPS VOICES",
"CULTURE & ARTS": "ARTS & CULTURE",
"FIFTY": "MISCELLANEOUS",
"GOOD NEWS": "MISCELLANEOUS"})

In [12]:
# one hot encoding using keras tokenizer and pad sequencing
X = final_df['length_of_news']
encoder = LabelEncoder()
y = encoder.fit_transform(final_df['category_merged'])
print("shape of input data: ", X.shape)
print("shape of target variable: ", y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
tokenizer = Tokenizer(num_words=10000, oov_token='<00V>') 
tokenizer.fit_on_texts(X_train) # build the word index


shape of input data:  (209527,)
shape of target variable:  (209527,)


In [13]:
# padding X_train text input data
train_seq = tokenizer.texts_to_sequences(X_train) # converts strinfs into integer lists
train_padseq = pad_sequences(train_seq, maxlen=130) # pads the integer lists to 2D integer tensor 

# padding X_test text input data
test_seq = tokenizer.texts_to_sequences(X_test)
test_padseq = pad_sequences(test_seq, maxlen=130)



In [14]:
word_index = tokenizer.word_index
max_words = 80000  # total number of words to consider in embedding layer
total_words = len(word_index)
num_classes = len(final_df['category_merged'].unique())
# Convert the labels to categorical
maxlen = 130 # max length of sequence 
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
# y_train = to_categorical(y_train, dtype ="uint8")
# y_test = to_categorical(y_test, dtype ="uint8")

print("Length of word index:", total_words)
# print(X_train.shape)

Length of word index: 157521


In [15]:
print(np.shape(X_train))
print(y_train.shape)
from tensorflow.keras.layers import Input
input_layer = Input(shape=(maxlen,))
# input_layer = Input(shape=(20,))

(167621,)
(167621, 27)


In [16]:
from tensorflow.keras.regularizers import l1_l2
model_3 = Sequential()
model_3.add(Embedding(total_words, 100, input_length=maxlen))
model_3.add(Bidirectional(LSTM(64, dropout=0.1, recurrent_dropout=0.10, activation='tanh', return_sequences=True)))
model_3.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.20, activation='tanh', return_sequences=True)))
model_3.add(Conv1D(72, 3, activation='relu'))
model_3.add(MaxPooling1D(2))
model_3.add(SimpleRNN(64, activation='tanh', dropout=0.2, recurrent_dropout=0.20, return_sequences=True))
model_3.add(GRU(64, recurrent_dropout=0.20, recurrent_regularizer=l1_l2(l1=0.01, l2=0.01)))
model_3.add(Dropout(0.2))
model_3.add(Dense(27, activation='sigmoid'))
model_3.summary()

c:\Users\DhanushDevadiga\Downloads\NEWS-Classifier-main\NEWS-Classifier-main\myenv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential

# Compile the model
model_3.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# Setup EarlyStopping callback
earlystopping = EarlyStopping(monitor='val_accuracy',
                              patience=5,
                              verbose=1,
                              mode='max')

# Setup ModelCheckpoint callback with .keras extension
checkpointer = ModelCheckpoint(filepath='bestmodel.keras',
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True)

callback_list = [checkpointer, earlystopping]

# Fit model to the data
history3 = model_3.fit(train_padseq, y_train, 
                       batch_size=64, 
                       epochs=20, 
                       validation_split=0.2,
                       shuffle=True,
                       callbacks=callback_list)

# Evaluate the model
test_loss3, test_acc3 = model_3.evaluate(test_padseq, y_test, verbose=0)
print("Test loss and accuracy:", test_loss3, test_acc3)

Epoch 1/20
2096/2096 ━━━━━━━━━━━━━━━━━━━━ 0s 577ms/step - accuracy: 0.2508 - loss: 3.4614
Epoch 1: val_loss improved from inf to 2.13317, saving model to bestmodel.keras
2096/2096 ━━━━━━━━━━━━━━━━━━━━ 1357s 617ms/step - accuracy: 0.2508 - loss: 3.4610 - val_accuracy: 0.4180 - val_loss: 2.1332
Epoch 2/20
2096/2096 ━━━━━━━━━━━━━━━━━━━━ 0s 629ms/step - accuracy: 0.4606 - loss: 2.0349
Epoch 2: val_loss improved from 2.13317 to 1.83400, saving model to bestmodel.keras
2096/2096 ━━━━━━━━━━━━━━━━━━━━ 1384s 660ms/step - accuracy: 0.4606 - loss: 2.0349 - val_accuracy: 0.5316 - val_loss: 1.8340
Epoch 3/20
 779/2096 ━━━━━━━━━━━━━━━━━━━━ 11:58 546ms/step - accuracy: 0.5469 - loss: 1.7613

In [ ]:
model_3.save("the_model.h5")

In [ ]:
model_3.save_weights("the_weight.h5")

In [ ]:
import pickle
pickle.dump(encoder, open('encoder_kaggle.pkl', 'wb'))
pickle.dump(tokenizer, open('tokenizer_kaggle.pkl', 'wb'))